In [142]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

DATA PREPROCESSION

In [143]:
legacymaleplayers = pd.read_csv('legacymaleplayers.csv', low_memory = False, na_values = '-')
players = pd.read_csv('players_22-1.csv', low_memory = False, na_values = '-')

In [144]:
legacymaleplayers

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [145]:
players

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [146]:
legacymaleplayers.shape

(161583, 110)

In [147]:
legacymaleplayers.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,...,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,...,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,34928.608856,2.559318,0.0,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,...,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,...,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,...,50.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,31.000000
50%,220621.000000,19.000000,2.0,66.000000,70.000000,7.250000e+05,4000.000000,25.000000,181.000000,75.000000,...,59.000000,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000,41.000000
75%,236958.000000,21.000000,2.0,70.000000,75.000000,1.800000e+06,10000.000000,28.000000,186.000000,80.000000,...,66.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000,46.000000
max,271817.000000,23.000000,2.0,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,...,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000,68.000000


In [148]:
legacymaleplayers.shape

(161583, 110)

In [149]:
legacymaleplayers.drop(['player_id','player_url','fifa_version','fifa_update_date','player_face_url','fifa_update','dob','short_name', 'long_name', 'league_id','league_name','club_team_id','club_name','club_jersey_number','club_loaned_from','club_joined_date','club_contract_valid_until_year','nationality_id','nationality_name','nation_team_id','nation_jersey_number','real_face','body_type','release_clause_eur','player_tags','player_traits','mentality_composure', 'work_rate'], axis = 1, inplace = True)

In [150]:
legacymaleplayers

,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_position,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,CF,93,95,100500000.0,550000.0,27,169,67,1.0,CF,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,"LW, LM",92,92,79000000.0,375000.0,29,185,80,1.0,LW,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,1.0,SUB,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,ST,90,90,52500000.0,275000.0,32,195,95,1.0,ST,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,GK,90,90,63500000.0,300000.0,28,193,92,1.0,GK,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,RW,46,61,110000.0,700.0,18,180,73,1.0,RES,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,LB,46,58,110000.0,750.0,19,188,83,1.0,RES,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,CDM,46,58,110000.0,500.0,19,181,73,1.0,RES,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,"RW, LW",46,70,150000.0,500.0,17,175,68,1.0,RES,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


In [151]:
players.shape

(19239, 110)

In [152]:
players.columns

Index(['sofifa_id', 'player_url', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age',
       ...
       'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url',
       'club_flag_url', 'nation_logo_url', 'nation_flag_url'],
      dtype='object', length=110)

FEATURE ENGINEERING

In [153]:
l = []
l_less = []
for i in legacymaleplayers.columns:
    if ((legacymaleplayers[i].isnull().sum()<(0.3 *(legacymaleplayers.shape[0])))):
        l.append(i)
    else:
        l_less.append(i)

In [154]:
l

['player_positions',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'age',
 'height_cm',
 'weight_kg',
 'league_level',
 'club_position',
 'preferred_foot',
 'weak_foot',
 'skill_moves',
 'international_reputation',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_reactions',
 'movement_balance',
 'power_shot_power',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'power_long_shots',
 'mentality_aggression',
 'mentality_interceptions',
 'mentality_positioning',
 'mentality_vision',
 'mentality_penalties',
 'defending_marking_awareness',
 'defending_standing_tackle',
 'defending_sliding_tackle',
 'goalkeeping_diving',
 'goalkeeping_handli

In [155]:
l_less

['nation_position', 'goalkeeping_speed']

In [156]:
legacymaleplayers = legacymaleplayers[l]

In [157]:
legacymaleplayers

,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_position,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,CF,93,95,100500000.0,550000.0,27,169,67,1.0,CF,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,"LW, LM",92,92,79000000.0,375000.0,29,185,80,1.0,LW,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,1.0,SUB,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,ST,90,90,52500000.0,275000.0,32,195,95,1.0,ST,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,GK,90,90,63500000.0,300000.0,28,193,92,1.0,GK,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,RW,46,61,110000.0,700.0,18,180,73,1.0,RES,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,LB,46,58,110000.0,750.0,19,188,83,1.0,RES,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,CDM,46,58,110000.0,500.0,19,181,73,1.0,RES,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,"RW, LW",46,70,150000.0,500.0,17,175,68,1.0,RES,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


Separate Numeric and Non-Numeric Features

In [158]:
numeric_data = legacymaleplayers.select_dtypes(include = np.number)
non_numeric_data = legacymaleplayers.select_dtypes(include = ['object'])

In [159]:
imp = IterativeImputer(max_iter = 10, random_state = 0)
numeric_data = pd.DataFrame(np.round(imp.fit_transform(numeric_data)), columns = numeric_data.columns)

In [160]:
numeric_data

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,weak_foot,skill_moves,...,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93.0,95.0,100500000.0,550000.0,27.0,169.0,67.0,1.0,3.0,4.0,...,90.0,76.0,25.0,21.0,20.0,6.0,11.0,15.0,14.0,8.0
1,92.0,92.0,79000000.0,375000.0,29.0,185.0,80.0,1.0,4.0,5.0,...,81.0,85.0,22.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,90.0,90.0,54500000.0,275000.0,30.0,180.0,80.0,1.0,2.0,4.0,...,84.0,80.0,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0
3,90.0,90.0,52500000.0,275000.0,32.0,195.0,95.0,1.0,4.0,4.0,...,83.0,91.0,25.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
4,90.0,90.0,63500000.0,300000.0,28.0,193.0,92.0,1.0,4.0,1.0,...,20.0,37.0,25.0,25.0,25.0,87.0,85.0,92.0,90.0,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,180.0,73.0,1.0,3.0,2.0,...,42.0,54.0,23.0,21.0,25.0,9.0,13.0,13.0,12.0,7.0
161579,46.0,58.0,110000.0,750.0,19.0,188.0,83.0,1.0,3.0,2.0,...,31.0,31.0,50.0,51.0,45.0,6.0,14.0,8.0,13.0,14.0
161580,46.0,58.0,110000.0,500.0,19.0,181.0,73.0,1.0,2.0,2.0,...,40.0,37.0,36.0,45.0,50.0,8.0,9.0,7.0,14.0,9.0
161581,46.0,70.0,150000.0,500.0,17.0,175.0,68.0,1.0,3.0,2.0,...,44.0,63.0,19.0,17.0,14.0,13.0,12.0,14.0,7.0,13.0


In [161]:
sc = SimpleImputer(strategy = 'median')
scaled = sc.fit_transform(numeric_data)

In [162]:
scaled

array([[9.300e+01, 9.500e+01, 1.005e+08, ..., 1.500e+01, 1.400e+01,
        8.000e+00],
       [9.200e+01, 9.200e+01, 7.900e+07, ..., 1.500e+01, 1.400e+01,
        1.100e+01],
       [9.000e+01, 9.000e+01, 5.450e+07, ..., 1.100e+01, 5.000e+00,
        1.500e+01],
       ...,
       [4.600e+01, 5.800e+01, 1.100e+05, ..., 7.000e+00, 1.400e+01,
        9.000e+00],
       [4.600e+01, 7.000e+01, 1.500e+05, ..., 1.400e+01, 7.000e+00,
        1.300e+01],
       [4.600e+01, 6.300e+01, 1.100e+05, ..., 6.000e+00, 6.000e+00,
        1.400e+01]])

In [163]:
encoder = ce.BinaryEncoder(cols = non_numeric_data.columns)
non_numeric_data = encoder.fit_transform(non_numeric_data)
non_numeric_data.dropna(inplace = True)
non_numeric_data

,player_positions_0,player_positions_1,player_positions_2,player_positions_3,player_positions_4,player_positions_5,player_positions_6,player_positions_7,player_positions_8,player_positions_9,...,rb_8,gk_0,gk_1,gk_2,gk_3,gk_4,gk_5,gk_6,gk_7,gk_8
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
161579,0,0,0,0,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,1,0
161580,0,0,0,0,0,0,1,1,1,1,...,1,0,0,0,0,0,1,0,0,1
161581,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0


In [164]:
legacymaleplayers = pd.concat([numeric_data, non_numeric_data], axis = 1).reset_index(drop = True)

In [165]:
corr_matrix = legacymaleplayers.corr()['overall'].sort_values(ascending = False)

In [166]:
corr_matrix 

overall               1.000000
movement_reactions    0.845753
potential             0.695362
wage_eur              0.611958
value_eur             0.567398
                        ...   
club_position_4      -0.244451
club_position_3      -0.247054
rcm_0                -0.385012
cm_0                 -0.385012
lcm_0                -0.385012
Name: overall, Length: 311, dtype: float64

In [167]:
legacymaleplayers_usable = corr_matrix[:21]

In [168]:
legacymaleplayers_usable.index

Index(['overall', 'movement_reactions', 'potential', 'wage_eur', 'value_eur',
       'passing', 'attacking_short_passing', 'mentality_vision',
       'international_reputation', 'skill_long_passing', 'power_shot_power',
       'physic', 'age', 'skill_ball_control', 'dribbling', 'skill_curve',
       'power_long_shots', 'shooting', 'mentality_aggression',
       'attacking_crossing', 'skill_fk_accuracy'],
      dtype='object')

In [169]:
legacymaleplayers = legacymaleplayers[legacymaleplayers_usable.index]

In [170]:
legacymaleplayers

,overall,movement_reactions,potential,wage_eur,value_eur,passing,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,...,physic,age,skill_ball_control,dribbling,skill_curve,power_long_shots,shooting,mentality_aggression,attacking_crossing,skill_fk_accuracy
0,93.0,94.0,95.0,550000.0,100500000.0,86.0,89.0,90.0,5.0,76.0,...,63.0,27.0,96.0,96.0,89.0,88.0,89.0,48.0,84.0,90.0
1,92.0,90.0,92.0,375000.0,79000000.0,81.0,82.0,81.0,5.0,72.0,...,79.0,29.0,92.0,91.0,88.0,93.0,93.0,63.0,83.0,79.0
2,90.0,89.0,90.0,275000.0,54500000.0,83.0,86.0,84.0,5.0,76.0,...,64.0,30.0,90.0,92.0,85.0,90.0,86.0,47.0,80.0,83.0
3,90.0,85.0,90.0,275000.0,52500000.0,81.0,84.0,83.0,5.0,76.0,...,86.0,32.0,90.0,86.0,80.0,88.0,91.0,84.0,76.0,80.0
4,90.0,89.0,90.0,300000.0,63500000.0,33.0,42.0,20.0,5.0,41.0,...,62.0,28.0,31.0,33.0,25.0,25.0,29.0,29.0,25.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,39.0,61.0,700.0,110000.0,40.0,38.0,42.0,1.0,33.0,...,44.0,18.0,45.0,47.0,39.0,31.0,48.0,26.0,48.0,35.0
161579,46.0,42.0,58.0,750.0,110000.0,29.0,30.0,31.0,1.0,25.0,...,57.0,19.0,32.0,34.0,24.0,25.0,25.0,46.0,30.0,26.0
161580,46.0,50.0,58.0,500.0,110000.0,43.0,51.0,40.0,1.0,43.0,...,53.0,19.0,35.0,46.0,30.0,32.0,36.0,51.0,37.0,30.0
161581,46.0,45.0,70.0,500.0,150000.0,36.0,40.0,44.0,1.0,26.0,...,42.0,17.0,43.0,46.0,38.0,47.0,50.0,27.0,29.0,25.0


In [171]:
numeric_data = players.select_dtypes(include = np.number)
non_numeric_data = players.select_dtypes(include = ['object'])

In [172]:
imp = IterativeImputer(max_iter = 10, random_state = 0)
numeric_data = pd.DataFrame(np.round(imp.fit_transform(numeric_data)), columns = numeric_data.columns)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  for feat_idx in ordered_idx:


In [173]:
sc = SimpleImputer(strategy = 'median')
scaled = sc.fit_transform(numeric_data)

In [174]:
encoder = ce.BinaryEncoder(cols = non_numeric_data.columns)
non_numeric_data = encoder.fit_transform(non_numeric_data)
non_numeric_data.dropna(inplace = True)
non_numeric_data

,player_url_0,player_url_1,player_url_2,player_url_3,player_url_4,player_url_5,player_url_6,player_url_7,player_url_8,player_url_9,...,nation_logo_url_4,nation_logo_url_5,nation_flag_url_0,nation_flag_url_1,nation_flag_url_2,nation_flag_url_3,nation_flag_url_4,nation_flag_url_5,nation_flag_url_6,nation_flag_url_7
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,1,0,0,1,0,1,1,0,0,1,...,1,0,0,0,1,1,1,1,0,0
19235,1,0,0,1,0,1,1,0,0,1,...,1,0,0,1,0,0,0,1,0,1
19236,1,0,0,1,0,1,1,0,0,1,...,1,0,0,1,0,0,0,1,0,1
19237,1,0,0,1,0,1,1,0,0,1,...,1,0,0,1,0,0,0,1,0,1


In [175]:
players = pd.concat([numeric_data, non_numeric_data], axis = 1).reset_index(drop = True)

In [176]:
players = players[legacymaleplayers_usable.index]

In [177]:
#Separating the overall column to scale the rest of the data
y = legacymaleplayers['overall']
x = legacymaleplayers.drop('overall', axis = 1)

In [178]:
scale = StandardScaler()
scaled = scale.fit_transform(x)

In [179]:
x = pd.DataFrame(scaled, columns = x.columns)

Training and Testing the Models

In [180]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state = 42)

In [181]:
#LINEAR REGRESSION
l = LinearRegression()
l.fit(xtrain, ytrain)

#DECISION TREE REGRESOR
dtree = DecisionTreeRegressor()
dtree.fit(xtrain, ytrain)

#HARD VOTING
knn = KNeighborsRegressor()
sv = SVR()


# Creating a VotingRegressor with the specified models
voting_regressor = VotingRegressor(estimators=[
    ('dtree', dtree),
    ('knn', knn),
    ('sv', sv),
    ('l', l)
])


In [182]:
import pickle

In [183]:
voting_regressor.fit(xtrain, ytrain)
# #pkl.dump(voting, open('/content/' + voting._class_._name_ + '.pkl', 'wb'))
y_pred = voting_regressor.predict(xtest)
# print(voting_regressor(ytest, y_pred))



In [184]:
joblib.dump(voting_regressor, 'best_model.pkl')

['best_model.pkl']

In [185]:
print(f"""Voting Regressor Tree Mean Absolute Error = {mean_absolute_error(y_pred, ytest)},
        Voting Regressor Decision Tree Mean Squared Error = {mean_squared_error(y_pred, ytest)},
        Voting Regressor Decision Tree Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, ytest))},
        Voting Regressor Decision Tree R2 Score = {r2_score(y_pred, ytest)}
""")

Voting Regressor Tree Mean Absolute Error = 0.8068368292407616,
        Voting Regressor Decision Tree Mean Squared Error = 1.1720289913799802,
        Voting Regressor Decision Tree Root Mean Squared Error = 1.082602877965868,
        Voting Regressor Decision Tree R2 Score = 0.9745928051842577



In [186]:
y_pred = l.predict(xtest)
print(f"""Linear Mean Absolute Error = {mean_absolute_error(y_pred, ytest)},
        Linear Mean Squared Error = {mean_squared_error(y_pred, ytest)},
        Linear Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, ytest))},
        Linear R2 Score = {r2_score(y_pred, ytest)}
""")

Linear Mean Absolute Error = 1.6199328905142818,
        Linear Mean Squared Error = 4.4116538421340445,
        Linear Root Mean Squared Error = 2.1003937350254223,
        Linear R2 Score = 0.9020780063409388



In [187]:
y_pred = dtree.predict(xtest)
print(f"""Decision Tree Mean Absolute Error = {mean_absolute_error(y_pred, ytest)},
        Decision Tree Mean Squared Error = {mean_squared_error(y_pred, ytest)},
        Decision Tree Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, ytest))},
        Decision Tree R2 Score = {r2_score(y_pred, ytest)}
""")

Decision Tree Mean Absolute Error = 0.6562799764829657,
        Decision Tree Mean Squared Error = 1.6662128291611227,
        Decision Tree Root Mean Squared Error = 1.2908186662584031,
        Decision Tree R2 Score = 0.9665079567697022



In [188]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score

# Initialize KFold with 3 splits
cv = KFold(n_splits=3)

# Initialize XGBoost Regressor
xgbr = xgb.XGBRegressor()

# # Define the parameter grid for GridSearchCV
# PARAMETERS_xgb = {
#     "max_depth": [1, 2, 3, 4],
#     "min_child_weight": [2, 4, 16],
#     "subsample": [0.8, 1],
#     "colsample_bytree": [0.8, 1],
#     "learning_rate": [0.3, 0.1, 0.03],
#     "n_estimators": [100, 500, 1000]
# }

# # Initialize GridSearchCV with XGBoost Regressor
# model_gs = GridSearchCV(xgbr, param_grid=PARAMETERS_xgb, cv=cv, scoring="neg_mean_squared_error", n_jobs=-1)
# model_gs.fit(xtrain, ytrain)

# Make predictions on the test set

xgbr.fit(xtrain,ytrain)
y_pred = xgbr.predict(xtest)

# Print the evaluation metrics
# print("Best parameters found: ", model_gs.best_params_)
# print("Best cross-validation score (neg_mean_squared_error): ", model_gs.best_score_)
print("Mean Squared Error on test set: ", mean_squared_error(ytest, y_pred))
print("R^2 Score on test set: ", r2_score(ytest, y_pred))

Mean Squared Error on test set:  0.832544552372583
R^2 Score on test set:  0.9832024790450665


In [189]:
joblib.dump(xgbr, 'working_model1.pkl')

['working_model1.pkl']

In [190]:
voting_regressor = VotingRegressor(estimators=[
    ('dtree', dtree),
    ('xgb', xgbr),
    ('l', l)
])

voting_regressor.fit(xtrain, ytrain)
# #pkl.dump(voting, open('/content/' + voting._class_._name_ + '.pkl', 'wb'))
y_pred = voting_regressor.predict(xtest)

print(f"""Voting Regressor Tree Mean Absolute Error = {mean_absolute_error(y_pred, ytest)},
        Voting Regressor Decision Tree Mean Squared Error = {mean_squared_error(y_pred, ytest)},
        Voting Regressor Decision Tree Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, ytest))},
        Voting Regressor Decision Tree R2 Score = {r2_score(y_pred, ytest)}
""")

Voting Regressor Tree Mean Absolute Error = 0.8115335264683786,
        Voting Regressor Decision Tree Mean Squared Error = 1.2000926550526536,
        Voting Regressor Decision Tree Root Mean Squared Error = 1.0954874052460182,
        Voting Regressor Decision Tree R2 Score = 0.9743696113100747



TESTING

In [191]:
testing = players

In [192]:
x1 = testing.drop('overall', axis=1)
y1 = testing['overall']

In [193]:
xtrain,xtest,ytrain,ytest=train_test_split(x1,y1,test_size=0.2,random_state = 42)

In [194]:
xtrain = scale.fit_transform(xtrain)
xtest = scale.fit_transform(xtest)

In [195]:
testing = voting_regressor.predict(xtest)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [198]:
print(f"""Final Model Mean Absolute Error = {mean_absolute_error(testing, ytest)},
        Final Model Mean Squared Error = {mean_squared_error(testing, ytest)},
        Final Model Root Mean Squared Error = {np.sqrt(mean_squared_error(testing, ytest))},
        Final Model R2 Score = {r2_score(testing, ytest)}
""")

Final Model Mean Absolute Error = 1.0325493495987428,
        Final Model Mean Squared Error = 2.004842420740209,
        Final Model Root Mean Squared Error = 1.4159245815862542,
        Final Model R2 Score = 0.9478046432258262



In [199]:
with open('best_model2.pkl', 'wb') as model_file:
    pickle.dump(voting_regressor, model_file)

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scale, scaler_file)